In [34]:
import requests
# pip install requests

from bs4 import BeautifulSoup
# pip install beautifulsoup

import csv

from datetime import datetime

from multiprocessing import Pool

In [35]:
def get_html(url):
    r = requests.get(url) # Response
    return r.text         # возвращает html_code страницы

def get_all_links(html):
    soup = BeautifulSoup(html, 'lxml')
    
    tds = soup.find('table', id="currencies-all").find_all('td', class_="currency-name")
    
    links = []
    
    for td in tds:
        a = td.find('a').get('href')   # string
        link = 'https://coinmarketcap.com' + a         #/currencies/bitcoin/
        links.append(link)
    return links

In [36]:
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        h1 = soup.find('h1', class_="details-panel-item--name").text.strip()
    except:
        h1 = ''      # если нет объектов - оставить пустой
        
    #ALTERNATIVE     __h1
    #   h1 = soup.find('h1', class_="details-panel-item--name").text.strip()   - 
    
    try:
        price = soup.find('span', id="quote_price").text.strip()  
    except:
        price = ''
        
    #ALTERNATIVE      __price
    #   price = soup.find('span', id="quote_price").text_strip
    
    data = {'name' : h1,
            'price': price}       # создание словаря {"ключ": данные}
    
    return data




def make_all(url):
    html = get_html(url)
    data = get_page_data(html)
    write_csv(data)
    

In [37]:
def write_csv(data):
    with open('parsed_data_name.csv', 'a') as f:       # 'a' - append;   f - переменная
        writer = csv.writer(f)
        
        writer.writerow( (data['name'],
                          data['price']) )
        print(data['name'],"parsed")
    

In [ ]:
def main():
    
    start = datetime.now()
    
    url = 'https://coinmarketcap.com/all/views/all/'

    all_links = get_all_links( get_html(url) )
    
    
    
    with Pool(5) as p:
        p.map(make_all, all_links)
    
    end = datetime.now()
    
    total = end - start
    print(str(total))
    
if __name__ == '__main__':
    main()